In [1]:
import mmcv
from mmengine.config import Config
from mmdet.registry import DATASETS
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
import torch
import os
import pandas as pd
from pycocotools.coco import COCO
import numpy as np

In [2]:
# 모든 모듈 등록
register_all_modules()

In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/data/ephemeral/home/kjh/mmdetection/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')

root='../dataset/'

# dataset config 수정
cfg.test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CocoDataset',
        data_root=root,
        ann_file=root+'test.json',
        data_prefix=dict(img=''),
        test_mode=True,
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='Resize', scale=(512, 512), keep_ratio=True),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(
                type='PackDetInputs',
                meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor')
            )
        ]
    )
)

cfg.test_evaluator = dict(
    type='CocoMetric',
    ann_file=root + 'test.json',
    metric='bbox',
    format_only=False
)

cfg.model.roi_head.bbox_head.num_classes = 10

In [4]:
# 체크포인트 디렉토리
checkpoint_dir = '/data/ephemeral/home/kjh/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash'

# 디렉토리 내의 모든 .pth 파일 찾기
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')]

print("사용 가능한 체크포인트:")
for i, file in enumerate(checkpoint_files):
    print(f"{i+1}. {file}")

사용 가능한 체크포인트:
1. epoch_10.pth
2. epoch_12.pth
3. epoch_11.pth


In [5]:
# 사용할 체크포인트 파일 이름 지정
selected_checkpoint = 'epoch_12.pth' 

checkpoint_path = os.path.join(checkpoint_dir, selected_checkpoint)
print(f"사용할 체크포인트: {checkpoint_path}")

if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"지정된 체크포인트 파일을 찾을 수 없음: {checkpoint_path}")

사용할 체크포인트: /data/ephemeral/home/kjh/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash/epoch_12.pth


In [6]:
# 모델 초기화 및 체크포인트 로드
model = init_detector(cfg, checkpoint_path, device='cuda:0')

# 테스트 데이터셋 생성
dataset = DATASETS.build(cfg.test_dataloader.dataset)

# 추론 및 결과 생성
results = []
for i in range(len(dataset)):
    img_path = dataset.get_data_info(i)['img_path']
    result = inference_detector(model, img_path)
    results.append(result)

Loads checkpoint by local backend from path: /data/ephemeral/home/kjh/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash/epoch_12.pth
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [8]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []

# 파일 경로 확인 및 출력
ann_file = cfg.test_dataloader.dataset.ann_file
if not os.path.isabs(ann_file):
    ann_file = os.path.join(root, ann_file)
print(f"Annotation file path: {ann_file}")

if not os.path.exists(ann_file):
    raise FileNotFoundError(f"Annotation file not found: {ann_file}")

#coco = COCO(cfg.test_dataloader.dataset.ann_file)
coco = COCO(ann_file)

for i, result in enumerate(results):
    try:
        prediction_string = ''
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        
        if hasattr(result, 'pred_instances'):
            bboxes = result.pred_instances.bboxes
            scores = result.pred_instances.scores
            labels = result.pred_instances.labels
            
            for label, score, bbox in zip(labels, scores, bboxes):
                # 텐서를 파이썬 스칼라로 변환
                label = int(label.item())
                score = float(score.item())
                bbox = [float(coord.item()) for coord in bbox]
                prediction_string += f"{label} {score} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]} "
        else:
            print(f"Warning: No pred_instances for image {i}")
        
        prediction_strings.append(prediction_string)
        file_names.append(image_info['file_name'])
    except Exception as e:
        print(f"Error processing result {i}: {e}")
        print(f"Result content: {result}")

# 결과 출력
print(f"Processed {len(prediction_strings)} predictions")
print(f"First prediction string: {prediction_strings[0] if prediction_strings else 'None'}")
print(f"First file name: {file_names[0] if file_names else 'None'}")

Annotation file path: /data/ephemeral/home/dataset/test.json
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Processed 4871 predictions
First prediction string: 7 0.9996753931045532 215.85044860839844 53.913970947265625 457.0826416015625 494.3774719238281 7 0.9989771842956543 601.9973754882812 525.0150146484375 955.9736938476562 1020.44775390625 7 0.9939602613449097 123.98695373535156 414.270263671875 320.45916748046875 723.0819091796875 7 0.9915015697479248 449.7225341796875 603.190673828125 638.8082275390625 874.396728515625 7 0.9696589708328247 629.268310546875 321.23748779296875 878.060546875 621.3170166015625 1 0.9675265550613403 554.57373046875 118.35907745361328 763.947509765625 362.34375 7 0.9640569090843201 395.7598571777344 197.01260375976562 596.3663940429688 546.0523681640625 7 0.8986942172050476 565.39697265625 337.32818603515625 711.701171875 607.1838989257812 7 0.769944429397583 267.25689697265625 440.28997802734375 430.026123046875 584

In [11]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(checkpoint_dir, f'submission_{os.path.splitext(selected_checkpoint)[0]}.csv'), index=None)
print(submission.head())

                                    PredictionString       image_id
0  7 0.9996753931045532 215.85044860839844 53.913...  test/0000.jpg
1  5 0.8772043585777283 344.44500732421875 246.10...  test/0001.jpg
2  1 0.9861852526664734 80.3682861328125 270.1204...  test/0002.jpg
3  9 0.9141060709953308 187.94729614257812 258.96...  test/0003.jpg
4  0 0.74819016456604 199.99029541015625 257.9699...  test/0004.jpg


In [10]:
"""

# DataFrame 생성 시 컬럼 순서 지정
submission = pd.DataFrame({
    'image_id': file_names,
    'PredictionString': prediction_strings
})

# CSV 파일 저장 시 컬럼 순서 명시
submission.to_csv(
    os.path.join(checkpoint_dir, f'submission_{os.path.splitext(selected_checkpoint)[0]}.csv'), 
    index=None, 
    columns=['image_id', 'PredictionString']
)

print(submission.head())

"""


        image_id                                   PredictionString
0  test/0000.jpg  7 0.9996753931045532 215.85044860839844 53.913...
1  test/0001.jpg  5 0.8772043585777283 344.44500732421875 246.10...
2  test/0002.jpg  1 0.9861852526664734 80.3682861328125 270.1204...
3  test/0003.jpg  9 0.9141060709953308 187.94729614257812 258.96...
4  test/0004.jpg  0 0.74819016456604 199.99029541015625 257.9699...
